<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  








In [37]:
import pandas as pd
import numpy as np

#columns = ['Bankenclearing',	'ursprüngliche Mandanten-ID']#	Dokumentennummer	Zeitstempel	Vertrag	Vorname	Name	Produkt	ZREG	Vertragsverwaltungsuser	Service	PayNet-Nummer	Dokumentenschlüssel	Anzahl Seiten
columns = ['contract_no', 'payment_amount']

payment_id = pd.DataFrame(np.arange(1,100001))
contract_no = pd.DataFrame(np.random.randint(1000001, size=100001))
payment_amount = pd.DataFrame(np.random.sample(1000001)*50000)

print(np.round_(payment_amount, decimals = 2))


print (payment_id.shape, contract_no.shape, payment_amount.shape)

#pd.merge(payment_id, contract_no, payment_amount)



                0
0        20907.58
1        37785.02
2        11870.02
3        37977.73
4         2174.39
5         1908.84
6        27686.54
7        10233.37
8        17802.57
9        44151.22
10       20034.08
11       38130.66
12       42327.81
13        7395.15
14       42903.75
15       42607.06
16       30928.35
17       39410.80
18       33236.75
19       34488.05
20        8750.66
21       38944.89
22       31993.98
23       24388.25
24       20177.68
25       21854.19
26       26988.68
27       16759.06
28        2569.68
29       35759.22
...           ...
999971    3324.09
999972   21955.48
999973   18397.31
999974    5670.43
999975    5906.57
999976   43710.09
999977    2806.12
999978   46350.49
999979   16046.28
999980   31671.18
999981   46015.83
999982    8430.24
999983   18515.32
999984   44622.17
999985   18395.12
999986   44437.82
999987    8184.13
999988   39536.07
999989   16984.36
999990   22272.47
999991   42453.86
999992   48526.42
999993   47002.02
999994   3